In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
import os
from matplotlib.lines import Line2D
from collections import Counter
import math
from sklearn.decomposition import PCA
from aging.behavior.syllables import relabel_by_usage, compute_usage
%matplotlib inline
import datetime

In [2]:
# plot/colors definitions
cpath = '/n/groups/datta/win/longtogeny/code/notebooks/exploration/Dana'
data_loc=cpath+'/figs/'
try:
    os.mkdir(data_loc)
except FileExistsError:
    # directory already exists
    pass

sns.set_style('white')

In [3]:
def compute_usage(df):
    onsets = np.where(np.diff(df['syllables']) != 0)[0]
    usage = df.loc[df.index[onsets], 'syllables'].value_counts(normalize=True)
    return usage

In [4]:
# filter by syllable usage
def syll_index_above_threshold(mean_df, threshold = 0.01):
    syll_mean_usage = mean_df.groupby(['syllables']).mean()
    return syll_mean_usage[syll_mean_usage > threshold].index.to_list()

In [5]:
## for females

In [11]:
## upload data frame females
data_folder = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_02')
path = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_02/longtogeny_females.parquet')
df = pd.read_parquet(path)

In [10]:
df.head()

,experiment,file,syllables,date,uuid,age,session_name,subject_name,mouse


In [7]:
# calculate age for males
first= pd.Timestamp(year=2021, month=7, day=15)
age = (df.date-first).dt.days+21
df.age=age

In [8]:
df['mouse'] = df.subject_name.str[:5]

In [ ]:
# calculate mouse name for males and replace with 1-16)
df['mouse'] = df.subject_name.str[:5]
keep_mice = ['01_01',
 '01_02',
 '01_03',
 '01_04',
 '02_01',
 '02_02',
 '02_03',
 '02_04',
 '03_01',
 '03_02',
 '03_03',
 '03_04',
 '04_01',
 '04_02',
 '04_03',
 '04_04']
df = df[df['mouse'].isin(keep_mice)].reset_index()
df['mouse'] = df['mouse'].replace(to_replace=df['mouse'].unique(), value=list(range(1,len(df['mouse'].unique())+1)))

# define cage
df['cage'] = df.subject_name.str[:2]

In [ ]:
# calculate usage
musages = df.groupby(['age','mouse','uuid','experiment','subject_name','cage'], observed=True, sort=False).apply(compute_usage)
musage_mtx = pd.pivot_table(musages.reset_index(), index=['age', 'mouse','cage','experiment','uuid'], columns='syllables', values='proportion').fillna(0)
musage_mtx.write_parquet(data_folder / "longtogeny_musage_mtx"", use_pyarrow=True)
musage.write_parquet(data_folder / "longtogeny_musage"", use_pyarrow=True)


#musage_mtx.columns = musage_mtx.columns.droplevel(0)
#mdf_all = musage_mtx.copy()

#filter most used syllables just males
#include_syll = syll_index_above_threshold(musages)
#musage_mtx = musage_mtx[include_syll]